In [1]:
import numpy as np
import azureml
from azureml.core import Workspace, Run
from azureml.core.model import Model

print("Azure ML: ", azureml.core.VERSION)
ws = Workspace.from_config()
model = Model(ws, "model_Linear")

Azure ML:  0.1.59
Found the config file in: /data/home/aml/notebooks/AML_Workshop/aml_config/config.json
Interactive authentication successfully completed.


In [4]:
from azureml.core.conda_dependencies import CondaDependencies 
myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn")
#myenv.add_conda_package("sklearn")
#myenv.add_conda_package("keras")
#myenv.add_conda_package("tensorflow")
with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

    
from azureml.core.image import ContainerImage
image_config = ContainerImage.image_configuration(execution_script = "Score.py",
                                                  runtime = "python",
                                                  conda_file = "myenv.yml",
                                                  dependencies = ['min_max_scaler.pickle','model_Linear.pickle'],
                                                  description = "",
                                                  tags = {"type": "classification"})


from azureml.core.webservice import AciWebservice
aciconfig = AciWebservice.deploy_configuration(cpu_cores = 1, 
                                               memory_gb = 2, 
                                               tags = {"type": "classification"},
                                               description = '')

from azureml.core.webservice import Webservice

service = Webservice.deploy_from_model(deployment_config = aciconfig,
                                       image_config = image_config,
                                       models = [model], # this is the registered model object
                                       name = 'aci-linear-6',
                                       workspace = ws)
service.wait_for_deployment(show_output = True)
print(service.state)
print(service.scoring_uri)

Creating image
Image creation operation finished for image aci-linear-6:1, operation "Succeeded"
Creating service
Running....................................
SucceededACI service creation operation finished, operation "Succeeded"
Healthy
http://168.61.19.128:80/score


In [6]:
import Score

test_df = Score.getTestInput()  # [?, 26]
test_df = test_df[test_df['id']==20] # ? filtered by id
test_df = test_df.tail(5)

test_json = test_df.to_json(orient='split')

import requests
import json

headers = {'Content-Type':'application/json'}
resp = requests.post(service.scoring_uri, test_json, headers=headers)

print("POST to url", service.scoring_uri)
print("prediction:", resp.text)


POST to url http://168.61.19.128:80/score
prediction: "1"
